In [11]:
!pip install scikit_posthocs -q


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [19]:
import sqlite3
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ana_utils import convert_listlike_cols, db_path, font_size, font_size_large, df_to_latex_table
import json

In [20]:
METRIC = "r2_score"

In [21]:
def sec2str(seconds):
    if seconds < 60:
        return str(seconds) + "s"
    elif seconds < 60*60:
        return str(seconds // 60) + "min"
    else:
        return str(seconds // (60*60)) + "h"

conn = sqlite3.connect(db_path)
#df = pd.read_sql(f"SELECT model, search_space_hash, {METRIC}, data_config_hash, data_id, search_space_name FROM results ", conn)
df = pd.read_sql(f"SELECT results.model, {METRIC}, data_config_hash, data_id, json_extract(search_spaces.config, '$.fit_kwargs') as fit_kwargs FROM results join search_spaces on results.search_space_hash == search_spaces.hash_key", conn)
conn.close()
df = convert_listlike_cols(df)
#list(df[df["model"]=="AutoGluon"]["config"])[0]
df["Model"] = df[["model", "fit_kwargs"]].apply(lambda x: x["model"] if x["fit_kwargs"] is None else f"{x['model']} ({sec2str(json.loads(x['fit_kwargs'])['time_limit'])})", axis=1)
df = df.drop(["fit_kwargs"], axis=1)
df[df["model"] == "AutoGluon"]

,model,r2_score,data_config_hash,data_id,Model
205,AutoGluon,"[0.5589910051285314, 0.579542484201468, 0.5869...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (1min)
206,AutoGluon,"[0.5774340675119211, 0.5983437843344199, 0.616...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (5min)
207,AutoGluon,"[0.57571813641036, 0.5976286926715879, 0.60973...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (10min)
208,AutoGluon,"[0.5777667360788237, 0.5982454770967465, 0.608...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (30min)
209,AutoGluon,"[0.5785848663201216, 0.5976996235386514, 0.608...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,0,AutoGluon (1h)
...,...,...,...,...,...
1025,AutoGluon,"[0.9380148513564589, 0.9325516827743671, 0.928...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,20,AutoGluon (4h)
1026,AutoGluon,"[0.9985474588524893, 0.9988041769846859, 0.997...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,12,AutoGluon (4h)
1027,AutoGluon,"[0.8818214083188164, 0.8569967045917024, 0.882...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,15,AutoGluon (4h)
1028,AutoGluon,"[0.9417590400755006, 0.9451994939656829, 0.947...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,18,AutoGluon (4h)


In [22]:
df['Model'].value_counts()

Model
ExtraTrees                 39
GradientBoosting           36
AutoGluon (4h)             36
AdaBoost                   35
Bagging                    35
DecisionTree               35
Dummy                      35
CatBoost                   35
BayesianRidge              35
PassiveAggressive          35
AutoGluon (10min)          35
AutoGluon (30min)          35
AutoGluon (1h)             35
LightGBM                   35
KNN                        35
ElasticNet                 35
AutoGluon (1min)           35
SGD                        35
M5                         35
Huber                      35
RidgeRegressor             34
Lasso                      34
PartialLeastSquares        34
AutoGluon (5min)           34
LARS                       34
MLP                        34
GaussianProcess            32
LassoLars                  32
Tweedie                    29
XGBoost                    27
RandomForest               21
SupportVectorRegression    15
PolynomialRegression        6
Name

In [4]:
df["avg_metric"] = df[METRIC].apply(
    lambda x: sum(x) / len(x) if x else None
)
df

,model,r2_score,data_config_hash,data_id,Model,avg_metric
0,AdaBoost,"[-0.037778274858445204, -0.11735292262315156, ...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,30,AdaBoost,-0.355752
1,AdaBoost,"[0.935159342607676, 0.9128069052258785, 0.9163...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,28,AdaBoost,0.924055
2,AdaBoost,"[0.94135188120515, 0.9176192286309859, 0.89680...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,1,AdaBoost,0.917190
3,AdaBoost,"[0.41415412781051564, 0.2926531324530277, 0.33...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,10,AdaBoost,0.407703
4,AdaBoost,"[0.8771650717039693, 0.9237746056776517, 0.927...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,3,AdaBoost,0.906974
...,...,...,...,...,...,...
1067,MLP,"[0.6204535163608919, 0.9491253925724097, 0.777...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,11,MLP,0.781561
1068,MLP,"[0.008457138919903295, 0.00351891429729434, 0....",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,12,MLP,-5082.526888
1069,MLP,"[0.60760007967176, 0.60613145556253, 0.6452904...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,23,MLP,0.640120
1070,MLP,"[0.9965992738416575, 0.6238325313004675, 0.998...",2a5bb20661bcb64796e2f4f2eb847266e7feb6cedf57ab...,13,MLP,0.924301


In [5]:
# Name 
# Avg Normalized Performance
# Avg Normalized Performance (Without AG) 
# Avg Rank
# Avg Rank (Without AG) ~ To answer is the weakest AG already the best method
# Avg Absolute Inference Time per sample

In [16]:
groups = df.groupby(by=["data_config_hash", "data_id"])
group_collection = []
all_models = df["Model"].unique()

for (hash, data_id), group_df in groups:
    # Because if worse than zero we dont really care by how much
    group_df.loc[group_df["avg_metric"] < 0, "avg_metric"] = 0
    # Set Missing Entries to 0 - they likely crahsed
    group_df = group_df.set_index(group_df["Model"])
    group_df = group_df[~group_df.index.duplicated(keep="first")]
    group_df = group_df.drop(["Model"], axis=1)
    group_df = group_df.reindex(all_models, fill_value=0)
    group_df["Model"] = group_df.index
    group_df = group_df.reset_index(drop=True)
    
    maximum = group_df["avg_metric"].max()
    median = group_df["avg_metric"].median()
    if maximum <= 0.001:
        group_df["normalized_metric"] = 0 # If all fail, all fail simple as that
    else:
        group_df["normalized_metric"] = (group_df["avg_metric"] - median) / (maximum-median)
    assert group_df["normalized_metric"].max() < 1.2
    group_df["rank"] = group_df["avg_metric"].rank(method="dense", ascending=False)
    group_df["rounded_rank"] = group_df["avg_metric"].round(2).rank(method="dense", ascending=False)
    group_df["losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max())
    group_df["true losses(tied)"] = (group_df["rounded_rank"] == group_df["rounded_rank"].max()) & (group_df["rounded_rank"] != group_df["rounded_rank"].min())

    group_df_wo_ag = group_df[group_df["model"] != "AutoGluon"]
    maximum_wo_ag = group_df_wo_ag["avg_metric"].max()
    median_wo_ag = group_df_wo_ag["avg_metric"].median()
    if maximum_wo_ag <= 0.001:
        group_df["normalized_metric_wo_ag"] = 0 # If all fail, all fail simple as that
    else:
        group_df["normalized_metric_wo_ag"] = (group_df["avg_metric"] - median_wo_ag) / (maximum_wo_ag-median_wo_ag)
        
    #group_df["rank_wo_ag"] = group_df["avg_metric"].rank(ascending=False)

    gavg = (
        group_df.groupby("model", as_index=False)
          .agg(avg_metric=("avg_metric", "max"))
    )
    
    # 2) rank groups by that average
    gavg["rank_wo_ag"] = gavg["avg_metric"].rank(method="dense", ascending=False)
    gavg["rank_wo_ag_round"] = gavg["avg_metric"].round(2).rank(method="dense", ascending=False)
    rank_wo_ag_round = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].round(2).rank(method="dense", ascending=False)
    gavg["victories_wo_ag(ties)"] = (rank_wo_ag_round == 1)
    gavg["victories_wo_ag"] = gavg[gavg["model"]!="AutoGluon"]["avg_metric"].rank(method="dense", ascending=False) == 1
    
    
    # 3) merge back — no restructuring of df
    group_df = group_df.merge(gavg[["model", "rank_wo_ag", "rank_wo_ag_round", "victories_wo_ag", "victories_wo_ag(ties)"]], on="model", how="left")
    #print(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag"])
    if list(group_df[group_df["model"] == "AutoGluon"]["rank_wo_ag_round"])[0] > 1:
        print(maximum, gavg[gavg["model"] == "AutoGluon"][["rank_wo_ag_round", "avg_metric"]])
    
    autogluon_names = group_df[group_df["model"]=="AutoGluon"]["Model"]
    ag_max_perf = 0
    for ag_name in autogluon_names:
        ag_perf = group_df[group_df["Model"] == ag_name]["avg_metric"].iloc[0]
        group_df[f"victories_against_{ag_name}"] = group_df["avg_metric"] > ag_perf
        group_df[f"major_victories_against_{ag_name}"] = group_df["avg_metric"] > (ag_perf*1.01)
        if ag_perf > ag_max_perf:
            ag_max_perf = ag_perf
    group_df[f"victories_against_best_ag"] = group_df["avg_metric"] > ag_max_perf
    group_df[f"major_victories_against_best_ag"] = group_df["avg_metric"] > (ag_max_perf*1.01)
    

    group_df["failures"] = group_df["avg_metric"] <= 0.001    
    group_df = group_df.reset_index(drop=False)
    # TODO Victories against AG (x/y/z/W)
    # TODO Victories without AG
    # best_models = list(group_df[group_df["avg_metric"] >= maximum*0.999]["model"])
    # if not "AutoGluon" in best_models:
        
    #     print(maximum, best_models)
    #     print(group_df[group_df["model"] == "AutoGluon"]["avg_metric"])
    group_collection.append(group_df)
df_ = pd.concat(group_collection)
df_ = df_.groupby(by=["Model"]).agg({
    "avg_metric": "mean",
    "rank": "mean",
    "rounded_rank": "mean",
    "rank_wo_ag": "mean", # Without counting AG several times
    "rank_wo_ag_round": "mean",
    "normalized_metric": "mean",
    "normalized_metric_wo_ag": "mean",
    "victories_wo_ag": "sum",
    "victories_against_AutoGluon (5min)": "sum",
    "victories_against_AutoGluon (30min)": "sum",
    "victories_against_AutoGluon (1h)": "sum",
    "major_victories_against_AutoGluon (1h)": "sum",
    "victories_against_AutoGluon (4h)": "sum",
    "victories_against_best_ag": "sum",
    "major_victories_against_best_ag": "sum",
    "losses(tied)": "sum",
    "true losses(tied)": "sum",
    "failures": "sum",
})

df_.sort_values(by=["normalized_metric_wo_ag", "rank"])

0.9251837187272006    rank_wo_ag_round  avg_metric
2               2.0    0.902895
0.7481730144957809    rank_wo_ag_round  avg_metric
2               3.0    0.659344


,avg_metric,rank,rounded_rank,rank_wo_ag,rank_wo_ag_round,normalized_metric,normalized_metric_wo_ag,victories_wo_ag,victories_against_AutoGluon (5min),victories_against_AutoGluon (30min),victories_against_AutoGluon (1h),major_victories_against_AutoGluon (1h),victories_against_AutoGluon (4h),victories_against_best_ag,major_victories_against_best_ag,losses(tied),true losses(tied),failures
Model,,,,,,,,,,,,,,,,,,
Dummy,0.000000,27.314286,12.400000,22.514286,11.714286,-36.113634,-27.015707,1,0,0,0,0,0,0,0,35,34,35
SupportVectorRegression,0.060736,24.942857,11.057143,20.171429,10.371429,-35.625015,-26.603465,1,0,0,0,0,0,0,0,30,29,30
PolynomialRegression,0.090297,25.400000,11.542857,20.628571,10.857143,-35.359988,-26.443249,1,0,0,0,0,0,0,0,29,28,29
RandomForest,0.387966,18.057143,7.542857,13.371429,6.942857,-27.578026,-24.072842,2,1,1,1,1,1,0,0,16,15,16
Tweedie,0.374163,21.400000,8.828571,16.714286,8.142857,-30.408584,-22.937645,1,1,0,1,1,1,0,0,9,8,9
ExtraTrees,0.620209,11.542857,4.200000,7.285714,3.714286,-23.297030,-21.829378,8,2,3,3,3,3,2,1,6,5,6
AutoGluon (5min),0.727929,5.342857,1.885714,2.028571,1.285714,-22.515850,-21.091492,False,0,7,8,3,11,0,0,2,1,2
DecisionTree,0.613445,18.714286,7.771429,13.971429,7.114286,-9.340413,-8.567380,1,0,0,0,0,1,0,0,4,3,4
KNN,0.530540,18.885714,7.742857,14.114286,7.057143,-5.384069,-4.399366,1,0,0,0,0,1,0,0,2,1,2


In [10]:
df_["Model"] = df_.index

df_to_latex_table(
    df=df_[["Model", "normalized_metric_wo_ag", "normalized_metric", "rounded_rank", "true losses(tied)", "failures"]],
    filepath="./cache/ranking.tex",
    caption="Performance comparison of all evaluated methods against AutoGluon",
    label="tab:ranking",
    float_fmt=".2f",
)

'\\begin{table}[t]\n\\centering\n\\begin{tabular}{lrrrrr}\n\\toprule\n Model                   &   normalized\\_metric\\_wo\\_ag &   normalized\\_metric &   rounded\\_rank &   true losses(tied) &   failures \\\\\n\\midrule\n AdaBoost                &                      0.36 &                0.17 &           5.4  &                   3 &          4 \\\\\n AutoGluon (10min)       &                      0.92 &                0.82 &           1.66 &                   0 &          1 \\\\\n AutoGluon (1h)          &                      1    &                0.91 &           1.51 &                   0 &          1 \\\\\n AutoGluon (1min)        &                      0.21 &                0.07 &           2.26 &                   0 &          1 \\\\\n AutoGluon (30min)       &                      1    &                0.91 &           1.49 &                   0 &          1 \\\\\n AutoGluon (4h)          &                      0.95 &                0.85 &           1.66 &                  

In [8]:
# TODO 
# Get Names of outlier DS describe what happened (fair but minor, fair server but only two models better, all shit)

In [13]:
import scikit_posthocs as sp
import matplotlib.pyplot as plt

fig = sp.critical_difference_diagram(
    df.values,
    df.columns.tolist(),
    alpha=0.05,  # Nemenyi
)
plt.show()


TypeError: critical_difference_diagram() got an unexpected keyword argument 'alpha'